In [ ]:

import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, RobustScaler, Normalizer, MinMaxScaler
from pycaret.classification import *


In [6]:
# porcion de datos a entrenar
data_proportion = 0.2


In [ ]:
# Configuracion para ejecutar en Google Colab
# from pycaret.utils import enable_colab
# enable_colab()

# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
# dataset_path = 'drive/MyDrive/TFM'
# train_filepath=dataset_path+'/data_1_train.h5'
# test_filepath=dataset_path+'/data_1_test.h5'


In [ ]:
# Configuracion para ejecutar en local

DATASET_PATH = 'dataset'
TRAIN_FILEPATH = DATASET_PATH+'/data_1_train.h5'
TEST_FILEPATH = DATASET_PATH+'/data_1_test.h5'


In [ ]:
import h5py

def load_data(file_name):
    with h5py.File(file_name, 'r') as f:
        print(f.keys())
        print("X shape:", f['X'].shape)
        print("Y shape:", f['Y'].shape)
        print("Labels:", np.unique(f['Y']))
        return np.array(f['X']), np.array(f['Y'])


In [ ]:

X_data, Y_data = load_data(TRAIN_FILEPATH)


In [7]:
def get_reduced_data(X, Y, proportion=0.1):
    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(
        X, Y, train_size=proportion, stratify=Y.flatten())
    return X_train, y_train


In [8]:
x, y = get_reduced_data(X_data, Y_data, proportion=data_proportion)
print("X reduced shape:", x.shape)
print("Y reduced shape:", y.shape)
print("Size of new dataset:", np.unique(y.flatten(), return_counts=True))


X reduced shape: (2496, 1024, 16)
Y reduced shape: (2496, 1)
Size of new dataset: (array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13], dtype=uint8), array([192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192],
      dtype=int64))


In [9]:
def get_reduced_pca(x, y, transform_func=StandardScaler()):
    x_reduced = np.empty([x.shape[0], x.shape[1]])
    for idx, row in enumerate(x):
        row_std = transform_func.fit_transform(row)
        temp = PCA(n_components=1).fit_transform(row_std)
        x_reduced[idx] = temp.reshape(temp.shape[0])
    return pd.concat([
        pd.DataFrame(data=x_reduced),
        pd.DataFrame(data=y, columns=['modulation'])

    ], axis=1)


In [10]:
pca_df = get_reduced_pca(x, y)
pca_df


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,modulation
0,-1.320009,-2.047237,1.009786,0.889143,-2.263088,-0.310780,2.231530,-0.664441,-1.938369,0.575059,1.620945,-1.746835,-1.259238,1.271290,0.231095,-2.262010,0.279843,1.934413,-1.382390,-1.184879,1.663100,0.506524,-2.140272,0.054684,1.213551,-0.990280,-1.680275,1.113018,1.042827,-2.021090,-0.894933,1.644120,-0.321005,-1.747543,0.552484,0.867110,-1.701659,-3.492423,6.730185,0.498689,-6.603655,1.041925,5.479790,-3.640406,-3.851135,4.607709,1.750647,-5.366860,-0.302462,4.636853,-2.025252,-3.664980,2.784265,2.403423,-3.842565,-0.887156,3.240788,-0.473313,-3.100416,1.462528,2.125708,-2.251128,-1.476357,2.020300,0.448432,-1.699048,-0.033994,1.351460,-0.632783,-1.165129,0.337037,0.410090,-0.545446,-0.311056,-0.126872,-0.175114,0.295801,-0.775602,-0.760191,1.021399,0.373084,-1.498049,0.303907,2.071500,-1.396971,-2.003596,1.739714,1.314663,-2.967345,-0.397952,2.827732,-0.944263,-3.656713,2.183093,2.830986,-3.800052,-2.084542,4.579614,-0.278776,-5.039114,1.798512,4.773942,-3.684865,-3.222510,5.208413,1.471731,-6.145862,0.836551,5.472055,-3.466226,-5.311639,5.369004,3.157326,-6.630098,-0.909949,7.133697,-2.290610,-6.534763,4.438839,5.525431,-6.814589,-2.668224,7.846774,-0.165972,-7.755823,3.292674,6.314719,-5.750571,-4.492041,7.633398,1.810034,-8.268476,0.929183,7.927517,-4.854443,-6.522189,5.907696,2.103818,-3.949906,-0.718140,4.057573,-1.451697,-4.073577,2.661880,3.181579,-3.142826,-1.482168,3.726372,-0.094736,-3.668455,1.237472,3.312861,-2.681847,-2.342636,3.248789,0.619149,-3.457173,0.483771,3.267242,-1.663082,-2.552826,2.122484,1.797359,-3.367475,-0.183367,3.138880,-1.076356,-3.029749,1.575288,1.648877,-2.507256,-1.032173,2.514294,-0.206455,-2.760853,0.448467,1.652250,-1.067942,-1.330439,1.835200,0.214668,-1.865809,0.073359,1.128277,-0.472699,-0.728747,0.605237,0.252076,0.047515,-0.055325,-0.331948,-0.052904,0.208137,-0.006491,0.062058,-0.047417,0.038499,-0.334373,0.090255,0.308591,-0.080987,-0.352361,-0.249080,-0.148267,-0.396996,-0.321380,0.012038,-0.115659,0.098250,-0.071699,-0.003912,-0.641164,-0.619692,0.523538,-0.307921,-1.003491,0.111143,0.

In [11]:
print("training with PCA data ")
modulation_cls = setup(data=pca_df, target='modulation', use_gpu=True,
                       session_id=0, silent=True)
save_config('pca.pkl')
best = compare_models()
print(best)
evaluate_model(best)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.2347,0.6719,0.2308,0.2327,0.2269,0.1696,0.1706,642.9690
lightgbm,Light Gradient Boosting Machine,0.2296,0.6785,0.2261,0.2155,0.2166,0.1644,0.1652,34.9500
et,Extra Trees Classifier,0.2186,0.6474,0.2157,0.2023,0.2022,0.1524,0.1534,0.9110
rf,Random Forest Classifier,0.2135,0.6584,0.2100,0.1867,0.1892,0.1465,0.1481,0.6860
knn,K Neighbors Classifier,0.1505,0.5571,0.1506,0.1097,0.1102,0.0801,0.0822,0.9080
gbc,Gradient Boosting Classifier,0.1505,0.5983,0.1483,0.1664,0.1514,0.0787,0.0792,364.1630
ada,Ada Boost Classifier,0.1271,0.5596,0.1177,0.1056,0.0797,0.0487,0.0570,6.7520
nb,Naive Bayes,0.1174,0.5222,0.1183,0.1126,0.1123,0.0435,0.0437,0.0500
dt,Decision Tree Classifier,0.1173,0.5218,0.1174,0.1190,0.1151,0.0437,0.0438,3.2850
lr,Logistic Regression,0.1025,0.4910,0.1025,0.0973,0.0972,0.0273,0.0274,1.7080


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [12]:
def get_reduced_tsne(x, y, transform_func=StandardScaler(), perplexity=100):
    x_reduced = np.empty([x.shape[0], x.shape[1]])
    for idx, row in enumerate(x):
        row_std = transform_func.fit_transform(row)
        temp = TSNE(n_components=1,
                    perplexity=perplexity).fit_transform(row_std)
        x_reduced[idx] = temp.reshape(temp.shape[0])
    return pd.concat([
        pd.DataFrame(data=x_reduced),
        pd.DataFrame(data=y, columns=['modulation'])

    ], axis=1)


In [13]:
tsne_df = get_reduced_tsne(x, y)
tsne_df


,0,1,2,3,4,5,6,7,8,9,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,modulation
0,10.858833,8.102809,-5.910376,-3.964908,10.332655,4.806425,-8.110294,-3.576454,8.739506,-5.852654,...,8.799119,4.833525,-8.800795,10.431771,5.404770,-6.201344,-4.078255,9.268750,4.972917,4
1,-0.138650,4.111567,-10.840371,9.552587,-6.575478,3.440100,-2.519476,3.264504,-2.454543,1.707543,...,1.803296,-3.770860,3.937659,-0.141712,4.834850,-4.401533,10.424917,-5.509928,10.399727,4
2,11.059196,-0.626030,-9.108818,8.256278,-11.259892,16.396185,-16.715492,16.076811,-16.689241,15.685677,...,15.501887,-16.185028,14.810109,-15.819775,14.382429,-15.300756,13.038624,-14.712568,12.543650,1
3,0.574786,3.884683,-6.938900,4.850959,-8.060697,14.378720,-11.554569,15.758930,-20.058533,17.019541,...,-18.643024,19.080034,-17.630348,19.691856,-17.000765,12.013070,-16.041359,11.660200,-15.695301,9
4,-13.701680,26.739740,1.068563,-29.915897,3.419245,20.007010,-5.037652,-13.982110,9.874806,12.820399,...,9.576455,13.579824,-20.785048,-9.653685,22.896822,-2.020969,-27.073500,5.892076,17.482904,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491,-13.232217,-15.157895,14.390079,2.937998,-19.626316,-2.148868,19.844118,-12.883460,-15.518072,14.404601,...,17.330078,-21.159761,-3.767478,21.533234,2.397620,-18.275208,11.748323,17.349945,-20.639473,2
2492,-1.272522,2.372869,-1.485323,1.732919,-2.624598,1.101551,-3.277024,0.626450,-2.821373,-0.229460,...,-9.789441,5.966029,-10.595999,7.535150,-15.401964,15.182033,-16.467308,17.533855,6.354031,9
2493,-14.209717,11.508614,-15.378940,22.358900,-16.961306,21.831072,-19.458385,21.405598,-21.103611,20.176937,...,15.026791,-10.986406,14.131021,-10.076434,4.988969,-9.433990,4.913028,-8.788749,-4.791603,12
2494,-3.704389,-5.673224,17.913965,10.425436,-21.652985,6.598493,21.917013,-10.808426,-17.111771,16.764521,...,16.291626,12.241323,-22.910076,-14.421219,19.474758,9.797529,-20.526155,6.383930,21.756063,12


In [14]:
print("training with TSNE data")
modulation_cls = setup(data=tsne_df, target='modulation',
                       use_gpu=True, session_id=0, silent=True)
save_config('tsne.pkl')
best = compare_models()
print(best)
evaluate_model(best)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.2347,0.6841,0.2310,0.2202,0.2202,0.1699,0.1709,0.7120
rf,Random Forest Classifier,0.2215,0.7002,0.2176,0.2068,0.2036,0.1554,0.1566,0.6380
catboost,CatBoost Classifier,0.2198,0.6961,0.2178,0.2276,0.2180,0.1541,0.1548,602.4750
lightgbm,Light Gradient Boosting Machine,0.2147,0.6838,0.2118,0.2141,0.2046,0.1481,0.1491,31.9940
gbc,Gradient Boosting Classifier,0.1964,0.6545,0.1941,0.1990,0.1933,0.1290,0.1296,357.9890
nb,Naive Bayes,0.1786,0.6834,0.1754,0.1723,0.1669,0.1095,0.1103,0.0450
dt,Decision Tree Classifier,0.1385,0.5333,0.1375,0.1403,0.1365,0.0666,0.0668,2.1890
ada,Ada Boost Classifier,0.1156,0.5735,0.1130,0.0859,0.0785,0.0405,0.0452,6.7230
lr,Logistic Regression,0.1065,0.5114,0.1052,0.1050,0.1036,0.0315,0.0316,2.4260
svm,SVM - Linear Kernel,0.0910,0.0000,0.0904,0.0939,0.0903,0.0150,0.0150,0.1580


ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=0, verbose=0,
                     warm_start=False)


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [15]:
def get_flatten_data(x, y,  transform_func=StandardScaler()):
    flatten_x = np.reshape(
        x, (x.shape[0], x.shape[1]*x.shape[2]))
    df = pd.concat([
        pd.DataFrame(data=flatten_x),
        pd.DataFrame(data=y, columns=['modulation'])

    ], axis=1)
    return df


In [16]:
flatten_df = get_flatten_data(x, y)
flatten_df


,0,1,2,3,4,5,6,7,8,9,...,16375,16376,16377,16378,16379,16380,16381,16382,16383,modulation
0,213,1386,-194,924,947,-32,-23,-1241,-1035,-2013,...,849,518,1572,-1415,-314,-473,1178,1584,837,4
1,-9,858,427,-119,270,-128,18,-258,96,-308,...,-253,137,1165,-726,-281,-469,1525,948,1164,4
2,465,2849,-485,-2107,-2965,213,-2302,-1096,765,2980,...,-2570,-1249,2353,-2475,-1376,-2652,658,2920,-760,1
3,672,2920,2069,3467,3736,-1986,1318,3922,4287,1623,...,-4228,-1589,-3922,-540,4554,1384,-4069,3697,-2260,9
4,2298,3915,4374,-1502,-4654,-855,-4888,749,-3134,-3545,...,262,1669,3612,3409,1422,-3102,-1979,-2826,3212,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491,459,2798,268,2981,-2071,2348,2055,2844,-1841,2247,...,1776,-1300,2216,-872,-3329,640,2848,3269,1768,2
2492,762,-221,568,-503,436,-530,1806,267,783,-1039,...,-1959,-4915,-342,3928,1873,-2358,-3438,119,-5264,9
2493,1050,-3615,2981,3403,948,-3331,-2953,-3313,-2259,3387,...,-3047,-3347,1081,3248,1672,274,-3284,3385,680,12
2494,207,856,496,1069,37,550,-1113,118,786,-772,...,-6021,2355,4508,2022,-5648,3375,3523,-3091,4835,12


In [17]:
print("training with original flatten data")
modulation_cls = setup(data=flatten_df, target='modulation', use_gpu=True,
                       session_id=0, silent=True)
save_config('flatten.pkl')
best = compare_models()
print(best)
evaluate_model(best)


,Description,Value
0,session_id,0
1,Target,modulation
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(2496, 16385)"
5,Missing Values,False
6,Numeric Features,16384
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


IntProgress(value=0, description='Processing: ', max=84)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:53:33
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Gradient Boosting Classifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.1780,0.6479,0.1754,0.1668,0.1654,0.1082,0.1090,1.5950
nb,Naive Bayes,0.1671,0.6374,0.1635,0.1796,0.1458,0.0947,0.0986,0.7670
dt,Decision Tree Classifier,0.1168,0.5215,0.1155,0.1187,0.1157,0.0429,0.0430,23.1010
ada,Ada Boost Classifier,0.1122,0.5793,0.1105,0.1095,0.1070,0.0375,0.0377,105.4690
svm,SVM - Linear Kernel,0.0991,0.0000,0.0986,0.1005,0.0955,0.0238,0.0239,1.6910
ridge,Ridge Classifier,0.0973,0.0000,0.0966,0.0821,0.0765,0.0212,0.0219,0.6010
lr,Logistic Regression,0.0933,0.5009,0.0920,0.0840,0.0854,0.0173,0.0174,4.1430
knn,K Neighbors Classifier,0.0893,0.5276,0.0876,0.0179,0.0278,0.0120,0.0158,4.1340
qda,Quadratic Discriminant Analysis,0.0727,0.4976,0.0730,0.0727,0.0713,-0.0048,-0.0048,3.9340
